<img style="float: left;" src="http://third.datacastle.cn/pkbigdata/master.other.img/8ef429f9-2032-47b8-bcc4-818fa9e41a25.png" width="50%">

In [11]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
op_train = pd.read_csv('../data/operation_train_new.csv')
tran_train = pd.read_csv('../data/transaction_train_new.csv')
tag_train = pd.read_csv('../data/tag_train_new.csv')

In [3]:
print(op_train.shape)
print(tran_train.shape)

(1460843, 20)
(264654, 27)


### 1.数据探索

In [4]:
def find_different_uid(op_uids, tran_uids):
    op_diff_uids = []
    for uid in op_uids:
        if uid not in tran_uids:
            op_diff_uids.append(uid)
        else:
            tran_uids.remove(uid)
    return op_diff_uids, tran_uids

def find_same_uid(src1_uids, src2_uids):
    same_uids = [uid for uid in src1_uids if uid in src2_uids]
    return same_uids

In [5]:
# 处理时间字符串
op_train['time'] = op_train['day'].apply(lambda x: "2018-08-%02d" % x) + ' ' + op_train['time']
op_train['timestamp'] = op_train['time'].apply(lambda x:time.mktime(time.strptime(x,'%Y-%m-%d %H:%M:%S')))

tran_train['time'] = tran_train['day'].apply(lambda x: "2018-08-%02d" % x) + ' ' + tran_train['time']
tran_train['timestamp'] = tran_train['time'].apply(lambda x:time.mktime(time.strptime(x,'%Y-%m-%d %H:%M:%S')))

In [6]:
# 根据UID进行排序 再根据timestamp进行排序 
op_train = op_train.sort_values(by=['UID', 'timestamp'],ascending=True).reset_index(drop=True)
tran_train = tran_train.sort_values(by=['UID', 'timestamp'],ascending=True).reset_index(drop=True)
tag_train = tag_train.sort_values(by=['UID'], ascending=True).reset_index(drop=True)
op_train_gb = op_train.groupby('UID')
tran_train_gb = tran_train.groupby('UID')

In [ ]:
# (该方法已被废弃)
# 根据UID进行分组，再根据timestamp进行排序 
op_train_gb = op_train.groupby('UID').apply(lambda x:x.sort_values(by='timestamp', ascending=True))
tran_train_gb = tran_train.groupby('UID').apply(lambda x:x.sort_values(by='timestamp', ascending=True))
# tag_train_gb = tag_train.groupby('UID', as_index=True)
op_train_gb = op_train_gb.groupby('UID')
tran_train_gb = tran_train_gb.groupby('UID')

#### 1.1对op进行分析

In [7]:
#  获取op和tran各自的uid
op_train_uids = [uid for uid, item in op_train_gb.groups.items()]
tran_train_uids = [uid for uid, item in tran_train_gb.groups.items()]

# 获取op和tran各自的tag
op_tag = tag_train[tag_train['UID'].isin(op_train['UID'])]
tran_tag = tag_train[tag_train['UID'].isin(tran_train['UID'])]

In [8]:
# 将op数据中的作假者与正常人分离开
op_train_gb_cnt = op_train_gb.count()
op_train_fake_stat = op_train_gb_cnt[op_train_gb_cnt.index.isin(op_tag['UID'][op_tag['Tag'] == 1])]
op_train_normal_stat = op_train_gb_cnt[op_train_gb_cnt.index.isin(op_tag['UID'][op_tag['Tag'] == 0])]

In [9]:
print(len(op_train_fake_stat))
op_train_fake_stat.head()

4277


,day,mode,success,time,os,version,device1,device2,device_code1,device_code2,device_code3,mac1,mac2,ip1,ip2,wifi,geo_code,ip1_sub,ip2_sub,timestamp
UID,,,,,,,,,,,,,,,,,,,,
10000,9,9,9,9,9,9,9,7,0,0,9,0,0,9,0,0,0,9,0,9
10019,10,10,10,10,10,10,10,8,0,0,10,0,0,10,0,0,0,10,0,10
10021,191,191,117,191,191,117,117,0,117,117,0,0,117,117,0,0,0,117,0,191
10041,8,8,8,8,8,8,8,7,8,8,0,6,2,8,0,1,8,8,0,8
10058,27,27,25,27,27,23,23,18,0,0,23,0,0,23,0,0,0,23,0,27


In [10]:
print(len(op_train_normal_stat))
op_train_normal_stat.head()

25451


,day,mode,success,time,os,version,device1,device2,device_code1,device_code2,device_code3,mac1,mac2,ip1,ip2,wifi,geo_code,ip1_sub,ip2_sub,timestamp
UID,,,,,,,,,,,,,,,,,,,,
10001,67,67,50,67,67,48,49,39,48,48,1,5,43,50,1,2,48,50,1,67
10002,11,11,11,11,11,10,10,8,10,10,0,4,6,10,0,0,6,10,0,11
10003,15,15,15,15,15,15,15,14,15,15,0,0,15,15,0,14,0,15,0,15
10004,34,34,34,34,34,34,34,33,34,34,0,9,25,34,0,16,14,34,0,34
10006,7,7,4,7,7,4,4,3,4,4,0,0,4,4,0,0,4,4,0,7
